In [8]:
from keras.models import load_model
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

In [2]:
model = load_model('./simple_rnn_imdb.h5')
model.summary()

2025-05-17 19:28:22.305711: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-05-17 19:28:22.306389: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-05-17 19:28:22.306478: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
I0000 00:00:1747490302.307549  303976 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1747490302.308987  303976 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (32, 500, 128)         │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (32, 128)              │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (32, 1)                │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,027 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [3]:
model.get_weights()

[array([[-0.4085313 ,  0.6511995 , -0.66831845, ...,  0.159736  ,
          0.57954925, -0.5807339 ],
        [-0.06343655,  0.08853569, -0.02664194, ..., -0.0492409 ,
          0.00814788, -0.06266458],
        [-0.02274999,  0.10068136, -0.07981072, ...,  0.05501684,
          0.05334917, -0.05065951],
        ...,
        [-0.01735821, -0.06331033,  0.03049495, ...,  0.00759057,
         -0.02016005,  0.0136216 ],
        [ 0.0229443 ,  0.07240846,  0.02609745, ..., -0.00828356,
         -0.05128346,  0.05849531],
        [ 0.03244672, -0.06112115,  0.06652319, ..., -0.0159819 ,
          0.00856343,  0.00846184]], dtype=float32),
 array([[-0.04953005, -0.03271702, -0.05578732, ..., -0.00405387,
          0.01836833, -0.02541094],
        [ 0.04473641, -0.13821106, -0.02581307, ...,  0.12455356,
          0.06719266,  0.03191964],
        [ 0.13183406,  0.07221784,  0.12914678, ..., -0.01893231,
         -0.06264122, -0.12844695],
        ...,
        [-0.10203364, -0.05568112,  0.1

In [6]:
# Use the default parameters to keras.datasets.imdb.load_data
start_char = 1
oov_char = 2
index_from = 3



word_index = imdb.get_word_index()
inverted_word_index = dict(
    (i + 3, word) for (word, i) in word_index.items()
)

In [14]:
def encode_review(review):
    encd_review = [word_index.get(word, oov_char) for word in review]
    return encd_review

In [10]:
max_len = 500

In [22]:
def preprocess_review(review):
    #Encode text to onehot
    encd_review = encode_review(review)
    #padding seq
    seq = sequence.pad_sequences([encd_review] ,maxlen=max_len)

    return seq

In [34]:
def sentiment_predict(review):

    seq = preprocess_review(review)
    
    prediction = model.predict(seq)

    sentiment = 'Positive' if prediction[0][0] > 0.5 else 'Negative'
    
    return sentiment, prediction[0][0]

In [35]:
review_text = """is an absolute disaster from start to finish. 
    It's a complete mess with no coherent story to speak of. 
    The plot is riddled with tired clichés and laughable dialogues that are borderline cringe-worthy. 
    I honestly don't understand how anyone with even a shred of class could sit through this. 
    The acting is, at best, irritating, and at worst, utterly repulsive. 
    It feels more like watching a low-budget adult film rather than a legitimate movie."""
seq = preprocess_review(review_text)
seq

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   10,
         587,    2,    3, 3360,    2,    3,  500,  587, 1601, 2011, 1203,
         827,  960,    2, 1092,   10,  587,    3,  587,  827,  960, 1476,
           2, 1206, 1476, 1601, 1980,    2,  587,  827,    3, 1476,  827,
           2,  827, 1601,    2, 1206,   10, 3360,   10,  587, 2020,    2,
           2,    2,    2,    2,    2,    2,    2,  827,  755,  587,    2,
           3,    2, 1145, 1601, 1980, 1654, 2011,  960,  827,  960,    2,
        1980,  960,  587,  587,    2, 1989,   10,  827, 2020,    2, 3360,
        1601,    2, 1145, 1601, 2020,  960, 1476,  960, 3360,  827,    2,
         587,  827, 1601, 1476, 5132,    2,  827, 1601,    2,  587, 1654,
         960,    3, 2292,    2, 1601, 1206,    2,    2,    2,    2,    2,
           2,    2,    2, 2020,  960, 

In [37]:
sentiment, score = sentiment_predict(review_text)
print(f"Sentiment : {sentiment}")
print(f"Score : {score}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 19s 19s/step
Sentiment : Positive
Score : 0.531328022480011
